# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제

In [3]:
df1 = pd.DataFrame({"age" : ["baby", "baby", "baby", "baby"],
                   "income" : ["high", "high", "medium", "low"],
                   "student" : ["no", "yes", "no", "yes"],
                   "credit_rating" : ["fair", "excellent", "fair", "fair"],
                   "class_buys_computer" : ["no", "no", "no", "no"]})

In [4]:
pd_data = pd_data.append(df1, ignore_index=True)

# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [46]:

def get_gini(df, label):
   
    # 타겟변수의 gini 구하기
    target_class = list(df[label].unique())
    
    target_yes = len(df[df[label]==target_class[0]])
    target_no = len(df[df[label]!=target_class[0]])
    p = target_yes/len(df[label])
    q = 1-p
    
    gini = 1 - p**2-q**2
    
    return  gini

In [47]:
get_gini(pd_data,'class_buys_computer')

0.5

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [48]:

import itertools # 변수의 모든 클래스 조합을 얻기 위해 itertools 불러오기

# 클래스가 4개 이상일 때

def get_binary_split(df,attribute):
    
    pool = df[attribute].unique()     # 변수의 클래스를 pool로 받음
    result = list()
    
    for i in range(0,len(pool)+1):
        result.extend(list(map(list,itertools.combinations(pool,i))))
    
    return result[1:-1]

In [49]:
get_binary_split(pd_data, "age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['baby'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['youth', 'baby'],
 ['middle_aged', 'senior'],
 ['middle_aged', 'baby'],
 ['senior', 'baby'],
 ['youth', 'middle_aged', 'senior'],
 ['youth', 'middle_aged', 'baby'],
 ['youth', 'senior', 'baby'],
 ['middle_aged', 'senior', 'baby']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [52]:
def get_attribute_gini_index(df, attribute, label):
    # get_attribute_gini_index(pd_data, "age", "class_buys_computer")
    
    class_values = list(df[attribute].unique())    # ('youth','middle_age','senior')
    key = get_binary_split(df,attribute)
    gini = list()
    
    for i in range(0,len(class_values)):
        D1 = len(df[df[attribute]==class_values[i]])/len(df)
        D2 = 1-D1
        # get_gini도 이용해서 gini 계산
        gini = D1 * get_gini(df[df[attribute]==class_values[i]],label) + D2 * get_gini(df[df[attribute]!=class_values[i]],label) 
        print(gini)
    
    result = dict(zip(key,gini)) 
    
    return result

In [53]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

4
0.49230769230769234
4
0.3571428571428571
4
0.49230769230769234
4
0.35714285714285726


TypeError: zip argument #2 must support iteration

{('youth',): 0.49230769230769234,

 ('middle_aged',): 0.3571428571428571,
 
 ('senior',): 0.49230769230769234,
 
 ('baby',): 0.3571428571428571,
 
 ('youth', 'middle_aged'): 0.4444444444444444,
 
 ('youth', 'senior'): 0.5,
 
 ('youth', 'baby'): 0.345679012345679,
 
 ('middle_aged', 'senior'): 0.345679012345679,
 
 ('middle_aged', 'baby'): 0.5,
 
 ('senior', 'baby'): 0.4444444444444444,
 
 ('youth', 'middle_aged', 'senior'): 0.3571428571428571,
 
 ('youth', 'middle_aged', 'baby'): 0.49230769230769234,
 
 ('youth', 'senior', 'baby'): 0.3571428571428571,
 
 ('middle_aged', 'senior', 'baby'): 0.49230769230769234}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [13]:

my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - ('youth', 'baby') : 0.345679012345679


Min - ('youth', 'baby') : 0.345679012345679

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [204]:
##문제1 답안
get_binary_split(pd_data, "income")

# high로 나누는게 제일 좋다.

[['medium'],
 ['high'],
 ['low'],
 ['medium', 'high'],
 ['medium', 'low'],
 ['high', 'low']]

In [205]:
##문제2 답안
columns = list(pd_data)[:-1] # 타겟변수를 제외한 변수의 이름 받기
for i in columns: 
    print(i + ":")
    print(min(get_attribute_gini_index(pd_data, i, "class_buys_computer").items()))

## age가 가장 낮은 Gini index를 가지고 있으므로 age가 가장 중요한 변수이며, Gini index는 0.3571이다.

age:
(('baby',), 0.3571428571428571)
income:
(('high',), 0.4722222222222221)
student:
(('no',), 0.4444444444444444)
credit_rating:
(('excellent',), 0.49350649350649367)


In [206]:
##문제3 답안
pd_data_middle = pd_data.loc[pd_data["age"] == "middle_aged"]
for i in columns:
    print(i + ":")
    print(min(get_attribute_gini_index(pd_data_middle, i, "class_buys_computer").items()))
#age가 middle_aged인 경우 타겟 변수인 class_buys_computer의 값이 모두 yes로 일정하기 때문에 다른 변수들이 의미없다.

age:


ValueError: min() arg is an empty sequence

In [207]:
pd_data_notmiddle = pd_data.loc[pd_data["age"] != "middle_aged"]
columns = list(pd_data)[:-1]
for i in columns:
    print(i + ":")
    print(min(get_attribute_gini_index(pd_data_notmiddle, i, "class_buys_computer").items()))
#age가 senior 혹은 youth일 경우 student가 중요한 변수이며, 지니계수는 0.32 이다.

age:
(('baby',), 0.35714285714285715)
income:
(('high',), 0.35714285714285715)
student:
(('no',), 0.3673469387755103)
credit_rating:
(('excellent',), 0.43174603174603166)
